Ok, we have the data we need to do something here. There's a million ways out from here, and we need to try all of them. We need a solid test / train / predict / backtest pipeline in place. 

Which means we need a strategy. Let's do something brutally, impishly stupid to start. Let's stick with the vibes thinking, and implement an ETF end of day strategy. Training prompts will be a list of headlines. For training responses, we'll take the best performing broad market levered ETF from close to open, and that will be the expected response.

I've always been a bit beffudled by overnight levered ETF pricing, what is it tracking? Who's buying? I don't understand this stuff, but maybe zero will...

First, let's come up with the "buy" signals for each day, which we'll calculate as the best performing of the pool from close to open on that day.

In [ ]:
%run 02-markets.ipynb

In [28]:
import pandas as pd

tickers = [
    'TQQQ',
    'SPXL',
    'UDOW',
    'SQQQ',
    'SPXS',
    'SDOW'
]

ticker_days = {ticker: fetch_dailies(ticker) for ticker in tickers}

def best_overnight_performance(ticker_days, date):
    best_performance = float('-inf')
    best_ticker = None
    
    for ticker, df in ticker_days.items():
        # Ensure the date exists in the DataFrame
        if date in df.index:
            # Get the overnight return for the given date
            overnight_return = df.loc[date, 'Overnight_Return']
            if overnight_return > best_performance:
                best_performance = overnight_return
                best_ticker = ticker
    
    return best_ticker

# calculate the best overnight performer for every day since 2022-01-01 and put this in a DataFrame
results = []
for date in pd.date_range('2022-01-01', '2022-12-31'):
    best_ticker = best_overnight_performance(ticker_days, date)
    if best_ticker:  # Ensure there is a best ticker for the date
        overnight_return_percentage = ticker_days[best_ticker].loc[date, 'Overnight_Return'] * 100
        results.append({'Date': date, 'Ticker': best_ticker, 'Overnight_Return_Percentage': overnight_return_percentage})

results_df = pd.DataFrame(results)

print('Distribution of best overnight performers')
# print the distribution of best overnight performers
print(results_df['Ticker'].value_counts())


Distribution of best overnight performers
Ticker
SQQQ    97
TQQQ    72
UDOW    32
SDOW    28
SPXS    13
SPXL     9
Name: count, dtype: int64


Ok, we have the best performing overnight return for every day since 2022-01-01. Now we need to get the headlines for each day, and then we can start to build a model. We will use only one data source for now, we can do watercolors later. Start with bloomberg.

In [ ]:
# TODO: get bloomberg headlines for each day
# TODO: get the best performing overnight return for each day
# TODO: put this in a DataFrame